In [1]:
import railtracks as rt
from pydantic import BaseModel, Field
from copy import deepcopy

In [2]:
gpt_4o = rt.llm.OpenAILLM("gpt-4o")


class AnswerPrediction(BaseModel):
    quality_answer: bool = Field(
        ..., description="The quality of the answer that was provided by the model."
    )


system_c_o_t = rt.llm.SystemMessage(
    "You are extremley intellegnt reviewer of information that ponders the provided question and context to provide a simple one paragraph plan where you discuss you thinking process and how you would go about answering the question, without explictely answer the question."
)
system_answer = rt.llm.SystemMessage(
    "You are a helpful assistant that works tirelessly to answer a question using the provided context as a guide to answer the question."
)
system_answer_reviewer = rt.llm.SystemMessage(
    "You are a reviewer of the plan that was provided by the model and will determine if the plan is feasible and if it will accomplish the task that was asked of you. Be harsh and clear when the plan is not adequete."
)


COTNode = rt.library.terminal_llm("COT", system_message=system_c_o_t, llm_model=gpt_4o)
AnswerNode = rt.library.terminal_llm(
    "Answerer", system_message=system_answer, llm_model=gpt_4o
)
ReviewerNode = rt.library.structured_llm(
    output_schema=AnswerPrediction,
    name="AnswerReviewer",
    system_message=system_answer_reviewer,
    llm_model=gpt_4o,
)


async def COTLLM(message_history: rt.llm.MessageHistory, number_trails: int = 4):
    original_message_history = deepcopy(message_history)
    for _ in range(number_trails):
        cot_response = await rt.call(COTNode, user_input=message_history)

        message_history.append(rt.llm.AssistantMessage("My plan:" + cot_response))

        answer_response = await rt.call(ReviewerNode, user_input=message_history)
        if not answer_response.quality_answer:
            continue
        else:
            break

    original_message_history.append(
        rt.llm.UserMessage("Contextual Plan: " + cot_response)
    )
    response = await rt.call(AnswerNode, user_input=original_message_history)

    return cot_response, response


ChainOfThought = rt.library.function_node(COTLLM)

In [3]:
message_history = rt.llm.MessageHistory([rt.llm.UserMessage("")])
with rt.Session() as runner:
    response = await rt.call(ChainOfThought, message_history=message_history)

print("COT plan: " + response.answer[0])
print("-------------------")
print("Answer: " + response.answer[1])

[+2.300  s] RT          : INFO     - START CREATED COTLLM
[+2.301  s] RT          : INFO     - COTLLM CREATED COT
[+8.928  s] RT          : INFO     - COT DONE
[+8.928  s] RT          : INFO     - COTLLM CREATED AnswerReviewer
[+9.603  s] RT          : INFO     - AnswerReviewer DONE
[+9.606  s] RT          : INFO     - COTLLM CREATED COT
[+13.462 s] RT          : INFO     - COT DONE
[+13.478 s] RT          : INFO     - COTLLM CREATED AnswerReviewer
[+13.995 s] RT          : INFO     - AnswerReviewer DONE
[+14.011 s] RT          : INFO     - COTLLM CREATED COT
[+17.657 s] RT          : INFO     - COT DONE
[+17.659 s] RT          : INFO     - COTLLM CREATED AnswerReviewer
[+18.230 s] RT          : INFO     - AnswerReviewer DONE
[+18.230 s] RT          : INFO     - COTLLM CREATED COT
[+22.074 s] RT          : INFO     - COT DONE
[+22.076 s] RT          : INFO     - COTLLM CREATED AnswerReviewer
[+22.875 s] RT          : INFO     - AnswerReviewer DONE
[+22.878 s] RT          : INFO     - C

COT plan: My plan: When posed with a question absent of explicit context, my approach is to systematically decipher potential meanings and domains it might pertain to. Initially, I'd pinpoint any key phrases or vocabulary that could suggest specific themes or fields of study, such as technology, culture, or economics. Based on these clues, I'd map out various interpretations or scenarios the question might address, evaluating them through diverse lenses. This would involve drawing from a broad spectrum of knowledge — historical precedents, theoretical frameworks, or recent advancements — to create a cohesive narrative. By integrating different perspectives, my objective is to offer a thoughtful answer that encompasses the underlying complexities and nuances, ensuring the response is both comprehensive and adaptable to various contexts.
-------------------
Answer: Your plan outlines a structured approach to tackle questions that lack explicit context. Here's a step-by-step breakdown of 